In [1]:
from bs4 import BeautifulSoup as bs
import requests
url = 'http://sofifa.com/players?offset=0'

In [2]:
def soup_maker(url):
    r = requests.get(url)
    markup = r.content
    soup = bs(markup, 'lxml')
    return soup

In [3]:
x = soup_maker(url)

In [4]:
def find_top_players(soup):
    table = soup.find('table', {'class': 'table table-hover persist-area'})
    tbody = table.find('tbody')
    all_a = tbody.find_all('a', {'class': ''})
    #print(all_a)
    #for player in all_a:
        #if player['href'][0:7] == '/player/':
    #    print(player['href'][0:8])
    #print(player['href'])
    #print('Hello')
    return(['http://sofifa.com' + player['href'] for player in all_a if player['href'][0:8] == '/player/'])

In [5]:
h = find_top_players(x)

In [6]:
x

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<title>Players FIFA 18 May 24, 2018 SoFIFA</title>
<meta content="width=device-width" name="viewport"/>
<meta content="VGaFvm9Qm-qhCd2LCDi8QKWXNPSrVE2SJ2iT8c3vI1g" name="google-site-verification"/>
<meta content="" name="description"/>
<meta content="Khachin Borjigin" name="author"/>
<meta content="2009 - 2018 SoFIFA.com" name="copyright"/>
<link href="/launcher-icon-4x.png" rel="shortcut icon"/>
<link href="/launcher-icon-4x.png" rel="apple-touch-icon"/>
<link href="https://sofifa.com/players?hl=en-US" hreflang="en" rel="alternate"/>
<link href="https://sofifa.com/players?hl=fr-FR" hreflang="fr" rel="alternate"/>
<link href="https://sofifa.com/players?hl=de-DE" hreflang="de" rel="alternate"/>
<link href="https://sofifa.com/players?hl=es-ES" hreflang="es" rel="alternate"/>
<link href="https://sofifa.com/players?hl=it-IT" hreflang="it" rel="alternate"/>
<link href="http

In [7]:
h

['http://sofifa.com/player/158023',
 'http://sofifa.com/player/20801',
 'http://sofifa.com/player/190871',
 'http://sofifa.com/player/176580',
 'http://sofifa.com/player/167495',
 'http://sofifa.com/player/193080',
 'http://sofifa.com/player/192985',
 'http://sofifa.com/player/188545',
 'http://sofifa.com/player/183277',
 'http://sofifa.com/player/182521',
 'http://sofifa.com/player/178603',
 'http://sofifa.com/player/167664',
 'http://sofifa.com/player/155862',
 'http://sofifa.com/player/211110',
 'http://sofifa.com/player/200389',
 'http://sofifa.com/player/192119',
 'http://sofifa.com/player/177003',
 'http://sofifa.com/player/168542',
 'http://sofifa.com/player/153079',
 'http://sofifa.com/player/138956',
 'http://sofifa.com/player/1179',
 'http://sofifa.com/player/215914',
 'http://sofifa.com/player/202126',
 'http://sofifa.com/player/195864',
 'http://sofifa.com/player/194765',
 'http://sofifa.com/player/190460',
 'http://sofifa.com/player/189509',
 'http://sofifa.com/player/1849

In [43]:
from bs4 import BeautifulSoup as bs
import requests
import re

url = 'http://sofifa.com/players?offset=0'


fifa_stats = ['Crossing', 'Finishing', 'Heading Accuracy',
              'Short Passing', 'Volleys', 'Dribbling', 'Curve',
              'Free Kick Accuracy', 'Long Passing', 'Ball Control',
              'Acceleration', 'Sprint Speed', 'Agility', 'Reactions',
              'Balance', 'Shot Power', 'Jumping', 'Stamina', 'Strength',
              'Long Shots', 'Aggression', 'Interceptions', 'Positioning',
              'Vision', 'Penalties', 'Composure', 'Marking', 'Standing Tackle',
              'Sliding Tackle', 'GK Diving', 'GK Handling', 'GK Kicking',
              'GK Positioning', 'GK Reflexes']


def soup_maker(url):
    r = requests.get(url)
    markup = r.content
    soup = bs(markup, 'lxml')
    return soup


def find_top_players(soup):
    final_details = {}
    table = soup.find('table', {'class': 'table table-hover persist-area'})
    tbody = table.find('tbody')
    all_a = tbody.find_all('a')
    #print(all_a)
    result=[]
    for player in all_a:
        #print(player['href'])
        final_details['short_name'] = player.text
        #print(final_details)
        if player['href'][0:8] == '/player/':
            final_details.update(player_all_details('http://sofifa.com' + player['href']))
        dict2 = final_details.copy()
        result.append(dict2)
    return result


def find_player_info(soup):
    player_data = {}
    player_data['image'] = soup.find('img')['data-src']
    player_data['full_name'] = soup.find('h1').text.split(' (')[0]
    span = soup.find('span', attrs={'class': None}).text.strip()
    #span = soup.find('span').text.strip()
    #print(soup)
    #print(span)
    dob = re.search('(\(.*)\)', span).group(0)
    player_data['dob'] = dob.replace('(', '').replace(')', '')
    #print(span)
    infos = span.replace(dob + ' ', '').split(' ')
    #infos = span.split('Age')
    infos.append(infos[-1])
    infos[-2] = infos[-3]
    infos[-3] = infos[-4]
    infos[-4] = infos[-5][-3:-1]
    infos[-4] = infos[-4]+infos[-5][-1]
    infos[-5] = infos[-5][:-3]
    #print(infos)
    player_data['pref_pos'] = infos[:infos.index('Age')]
    #print(player_data['pref_pos'])
    player_data['age'] = int(infos[infos.index('Age') + 1: -2][0])
    player_data['height'] = int((infos[infos.index('Age') + 2: -1][0]).replace('cm', ''))
    player_data['weight'] = int((infos[infos.index('Age') + 3:][0]).replace('kg', ''))
    return(player_data)


def find_player_stats(soup):
    player_data = {}
    info = re.findall('\d+', soup.text)
    player_data['rating'] = int(info[0])
    player_data['potential'] = int(info[1])
    player_data['value'] = int(info[2])
    player_data['wage'] = int(info[3])
    return(player_data)

def find_player_stats(soup):
    player_data = {}
    info = re.findall('\d+', soup.text)
    #print(info[4])
    player_data['rating'] = int(info[0])
    player_data['potential'] = int(info[1])
    player_data['value'] = int(info[2])
    if len(info)== 5:
        player_data['wage'] = int(info[4])*1000
    else:
        player_data['wage'] = int(info[3])*1000
    return(player_data)

def find_player_secondary_info(soup):
    player_data = {}
    #print(soup)
    player_data['preff_foot'] = soup.find('label', text='Preferred Foot')\
        .parent.contents[2].strip('\n ')
    player_data['club'] = soup.find_all('ul')[1].find('a').text
    player_data['club_pos'] = soup.find('label', text='Position')\
        .parent.find('span').text
    player_data['club_jersey'] = soup.find('label', text='Jersey Number')\
        .parent.contents[2].strip('\n ')
    if soup.find('label', text='Joined'):
        player_data['club_joined'] = soup.find('label', text='Joined')\
            .parent.contents[2].strip('\n ')
    if soup.find('label', text='Contract Valid Until'):
        player_data['contract_valid'] = soup.find('label', text='Contract Valid Until').parent.contents[2].strip('\n ')
    if len(soup.find_all('ul')) > 2:
        player_data['country'] = soup.find_all('ul')[2].find('a').text
    return(player_data)


def find_fifa_info(soup):
    player_data = {}
    #print(soup[4])
    #divs_without_skill = soup[1].find_all('div', {'class': 'column col-3 mb-20'})[:3]
    divs_without_skill = soup[4].find_all('div', {'class': 'column col-3 mb-20'})[:3]
    #print(divs_without_skill)
    more_lis = [div.find_all('li') for div in divs_without_skill]
    lis = soup[0].find_all('li') + more_lis[0]
    #print(lis)
    for li in lis:
        for stats in fifa_stats:
            if stats in li.text:
                #print(stats.replace(' ', '_').lower())
                #print(li.text.split(' ')[0]).replace('\n', '')
                player_data[stats.replace(' ', '_').lower()] = int((li.text.split(' ')[0]).replace('\n', ''))
    #traits = soup[1].find('h4', text='Traits')
    traits = soup[4].find('h5', text='Traits')
    #print(traits)
    if traits:
        #print(traits.parent.find_all('li'))
        #player_data['traits'] = [li.text.replace('\xa0', '') for li in traits.parent.next_sibling.next_sibling.find_all('li')]
        player_data['traits'] = [li.text.replace('\xa0', '') for li in traits.parent.find_all('li')]
    specialities = soup[1].find('h4', text='Specialities')
    if specialities:
        #print('hello')
        player_data['specialities'] = [li.text.replace('\xa0', '') for li in
            specialities.parent.next_sibling.next_sibling.find_all('li')]
    return(player_data)


def player_all_details(url):
    all_details = {}
    #print(url)
    soup = soup_maker(url)
    player_info = soup.find('div', {'class': 'player'})
    #print(player_info)
    all_details.update(find_player_info(player_info))
    player_stats = soup.find('div', {'class': 'stats'})
    all_details.update(find_player_stats(player_stats))
    secondary_info = soup.find('div', {'class': 'teams'})
    all_details.update(find_player_secondary_info(secondary_info))
    fifa_info = soup.find_all('div', {'class': 'columns'})
    #print(fifa_info)
    all_details.update(find_fifa_info(fifa_info))
    return(all_details)

In [44]:
soup = soup_maker(url)
result=find_top_players(soup)

In [10]:
urls=['http://sofifa.com/player/158023',
 'http://sofifa.com/player/20801']

In [11]:
find_top_players(urls)

AttributeError: 'list' object has no attribute 'find'

In [13]:
type(soup)

bs4.BeautifulSoup

In [22]:
player_all_details('http://sofifa.com/player/158023')

Info size is 5


{'age': 30,
 'aggression': 48,
 'club': '',
 'club_jersey': '\r\n10',
 'club_joined': '\r\nJul 1, 2004',
 'club_pos': 'RS',
 'composure': 97,
 'contract_valid': '\r\n2021',
 'country': '',
 'dob': 'Jun 24, 1987',
 'full_name': 'L. Messi',
 'height': 170,
 'image': 'https://cdn.sofifa.org/1x/18/players/158023.png',
 'interceptions': 22,
 'penalties': 75,
 'positioning': 93,
 'potential': 94,
 'pref_pos': ['Lionel', 'Messi', '', 'CF', 'ST', 'RW'],
 'preff_foot': '\r\nLeft',
 'rating': 94,
 'traits': ['Finesse Shot',
  'Long Shot Taker',
  'Speed Dribbler',
  'Playmaker',
  'One Club Player',
  'Chip Shot'],
 'value': 118,
 'vision': 92,
 'weight': 72}

In [36]:
type(result[0])

dict

In [51]:
result[30]

{'age': 26,
 'aggression': 73,
 'club': '',
 'club_jersey': '\r\n17',
 'club_joined': '\r\nAug 30, 2015',
 'club_pos': 'RCM',
 'composure': 86,
 'contract_valid': '\r\n2023',
 'country': '',
 'dob': 'Jun 28, 1991',
 'full_name': 'K. De Bruyne',
 'height': 181,
 'image': 'https://cdn.sofifa.org/1x/18/players/192985.png',
 'interceptions': 60,
 'penalties': 77,
 'positioning': 86,
 'potential': 92,
 'pref_pos': ['Kevin', 'De', 'Bruyne', '', 'CAM', 'CM'],
 'preff_foot': '\r\nRight',
 'rating': 91,
 'short_name': 'CM',
 'traits': ['Leadership',
  'Early Crosser',
  'Long Passer',
  'Long Shot Taker',
  'Playmaker'],
 'value': 104,
 'vision': 92,
 'wage': 395000,
 'weight': 68}